In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
import sklearn
df = pd.read_csv(r'C:\Users\Lenovo\Desktop\bitirme\data\vader_dataset.csv')
df

,Artist,Title,Album,Date,Lyric,compound,neg,neu,pos,VADER sentiment
0,Ariana Grande,"​thank u, next","thank u, next",2018-11-03,thought i'd end up with sean but he wasn't a m...,0.9998,0.061,0.517,0.422,Positive
1,Ariana Grande,7 rings,"thank u, next",2019-01-18,yeah breakfast at tiffany's and bottles of bub...,0.9983,0.058,0.686,0.256,Positive
2,Ariana Grande,​God is a woman,Sweetener,2018-07-13,you you love it how i move you you love it how...,0.9982,0.010,0.759,0.231,Positive
3,Ariana Grande,Side To Side,Dangerous Woman,2016-05-20,ariana grande nicki minaj i've been here all ...,-0.6497,0.067,0.868,0.065,Negative
4,Ariana Grande,​​no tears left to cry,Sweetener,2018-04-20,right now i'm in a state of mind i wanna be in...,0.7234,0.113,0.748,0.139,Positive
...,...,...,...,...,...,...,...,...,...,...
1097,Taylor Swift,Teardrops on My Guitar (Live from SoHo),Live from SoHo - EP,2008-01-15,drew looks at me i fake a smile so he won't s...,0.9618,0.033,0.847,0.120,Positive
1098,Taylor Swift,Picture To Burn (Live from Clear Channel Strip...,Live From Clear Channel Stripped 2008,2008-07-28,state the obvious i didn't get my perfect fant...,-0.9980,0.302,0.633,0.064,Negative
1099,Taylor Swift,Rare Peer (Brenda Lee Tribute),Woman Walk The Line,2017-09-20,i'm curled up in my mother's bed staring inten...,0.9980,0.032,0.791,0.178,Positive
1100,Taylor Swift,Should’ve Said No (Live from Clear Channel Str...,Live From Clear Channel Stripped 2008,2008-06-28,it's strange to think the songs we used to sin...,0.8174,0.066,0.845,0.089,Positive


In [2]:
training_data = df.sample(frac=0.8, random_state=25) #pandas function sample
testing_data = df.drop(training_data.index)

In [3]:
len(training_data[training_data['VADER sentiment'] == "Positive"])

608

In [4]:
len(training_data[training_data['VADER sentiment'] == "Negative"])

274

In [5]:
len(testing_data[testing_data['VADER sentiment'] == "Positive"])

150

In [6]:
len(testing_data[testing_data['VADER sentiment'] == "Negative"])

70

The fact that the ratio of positive to negative in both training and test datasets is close increased the f1-score for the negative songs. 

In [7]:
#Using Tfidf model for vectorizing the lyrics
vectorizer = TfidfVectorizer(sublinear_tf=True, use_idf = True, stop_words = 'english')
train_vectors = vectorizer.fit_transform(training_data['Lyric'])
test_vectors = vectorizer.transform(testing_data['Lyric'])

In [8]:
import time
from sklearn import svm
from sklearn.metrics import classification_report
#Classification with SVM and kernel is linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, training_data['VADER sentiment'])
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1
# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(testing_data['VADER sentiment'], prediction_linear, output_dict=True)
print('positive: ', report['Positive'])
print('negative: ', report['Negative'])

Training time: 0.497599s; Prediction time: 0.124911s
positive:  {'precision': 0.8333333333333334, 'recall': 0.9666666666666667, 'f1-score': 0.8950617283950617, 'support': 150}
negative:  {'precision': 0.8913043478260869, 'recall': 0.5857142857142857, 'f1-score': 0.7068965517241379, 'support': 70}
